# Naive Bayes
4/3 First I tried naive bayes on the entire data set used in the pipeline project, runs allwed total, etc and got ~55% accuracy.  Using just delta win percentage gives ~57.5% - almost breaking even with the straight odds of just going with the higher rated team which is 58%.  Cross validation puts the odds exactly at what you'd expect ~58%

Naive Bayes goes down with increasing features (0.576)

BUT, if the features are then labelled (simply as True for greater than 0 and False otherwise, the CV produces 0.599 - 59%.  The first time it's been better in awhile!!

I used 60% test size

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc
from numpy import interp
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv("2019_all_delta_extra.csv")
df

,Date,Home_Win,Home,Away,Win Percent Delta,Streak Delta,Run Scored Delta,Run Allowed Delta,Run Delta Delta,Run Delta Percent Delta,Win Total Delta
0,thursday mar 28,W,OAK,LAA,0.000000,-2,11,14,-3,0.7857,0
1,friday mar 29,L,OAK,LAA,0.333333,2,15,10,5,1.0714,1
2,saturday mar 30,W,OAK,LAA,-0.250000,-2,11,14,-3,-0.1500,0
3,sunday mar 31,W,OAK,LAA,0.066667,2,13,12,1,0.1545,1
4,monday apr 1,W,SEA,LAA,0.583333,4,39,23,16,0.6214,4
...,...,...,...,...,...,...,...,...,...,...,...
2424,sunday apr 14,L,NYY,CHW,0.120879,2,12,-29,41,0.5459,2
2425,thursday jun 13,W,CHW,NYY,-0.136364,2,-75,50,-125,-0.4098,-9
2426,friday jun 14,W,CHW,NYY,-0.119403,4,-74,49,-123,-0.3972,-8
2427,saturday jun 15,L,CHW,NYY,-0.102941,6,-66,41,-107,-0.3382,-7


In [3]:
X = df.loc[:,df.columns[4:]]
X_bool = (X >= 0).values
y = df.loc[:,df.columns[1]].values

In [4]:
#create win/loss mask
wins = df.loc[:,'Home_Win'] == 'W'
losses = df.loc[:,'Home_Win'] != 'W'

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
scaler = StandardScaler()
X_transform = scaler.fit_transform(X)
le = LabelEncoder()
y_label = le.fit_transform(y)
le.transform(['W','L'])

array([1, 0], dtype=int64)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_bool, y_label, test_size=0.6, random_state=0)

In [7]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
print(f'Accuracy: {round(1-((y_test != y_pred).sum()/X_test.shape[0]),3)}')

Number of mislabeled points out of a total 1458 points : 632
Accuracy: 0.567


# Cross-Validation
Using 10 fold cross validation o nthe entire dataset gives 58% accuracy - equal to my naive assumption that winning teams will continue winning.  For some reason 10 fold cross validation on just the training set produces better results - ~60% accuracy.  I think that there are some outliers in the test set

In [21]:
scores = cross_val_score(estimator=gnb,
                             X=X,
                             y=y,
                             cv=10,
                             n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [0.58847737 0.61316872 0.57201646 0.51851852 0.5308642  0.59259259
 0.59670782 0.60082305 0.55967078 0.62396694]
CV accuracy: 0.580 +/- 0.033


In [22]:
scores = cross_val_score(estimator=gnb,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [0.65306122 0.56701031 0.59793814 0.63917526 0.60824742 0.48453608
 0.62886598 0.5257732  0.60824742 0.68041237]
CV accuracy: 0.599 +/- 0.056
